## The models

In [5]:
import argparse

from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, Flatten, ELU, Activation, Lambda, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.initializers import glorot_uniform
from random import randrange
from keras.utils.np_utils import to_categorical

#from common import load_data, load_data_hog, train_and_save_results, L2Penalty

seed=10102016

In [12]:
def get_model_1(learning):
        
    image_shape = (28, 28, 1)

    model = Sequential()
    model.add(Lambda(lambda x: x, input_shape=image_shape, output_shape=image_shape))
    
    model.add(Convolution2D(
        5, 
        5, 
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        border_mode="valid"))
    model.add(MaxPooling2D())
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(5, kernel_initializer=glorot_uniform(seed), activation='softmax'))

    optimizer = Adam(lr=learning)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    return model

def get_model_2(learning):
        
    image_shape = (28, 28, 1)

    model = Sequential()
    model.add(Lambda(lambda x: x, input_shape=image_shape, output_shape=image_shape))
    
    model.add(Convolution2D(
        5, 
        5, 
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        border_mode="valid"))
    model.add(MaxPooling2D())
    model.add(Activation('relu'))

    model.add(Convolution2D(
        10, 
        5, 
        kernel_initializer='random_uniform',
        bias_initializer='zeros',
        border_mode="valid"))
    model.add(MaxPooling2D())
    model.add(Activation('relu'))

    model.add(Flatten())
    model.add(Dense(5, kernel_initializer=glorot_uniform(seed), activation='softmax'))

    optimizer = Adam(lr=learning)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    return model

In [13]:
learning_rate = 0.01
model1 = get_model_1(learning_rate)
model2 = get_model_2(learning_rate)

print("Model 1 summary")
model1.summary()

print("Model 2 summary")
model2.summary()

/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, 5, kernel_initializer="random_uniform", bias_initializer="zeros", padding="valid")`
  del sys.path[0]
/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, 5, kernel_initializer="random_uniform", bias_initializer="zeros", padding="valid")`
/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, 5, kernel_initializer="random_uniform", bias_initializer="zeros", padding="valid")`


Model 1 summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 5)         130       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 5)         0         
_________________________________________________________________
activation_1 (Activation)    (None, 12, 12, 5)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 720)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 3605      
Total params: 3,735
Trainable params: 3,735
Non-trainable params: 0
__________________________________________________________

## Data

In [51]:
def load_dataset(file_train, file_test, size=1.0):
    data_train = np.load(file_train)
    data_test = np.load(file_test)
    
    xtr, ytr = data_train["inputs"], data_train["targets"].reshape((-1, 1))
    xte, yte = data_test["inputs"], data_test["targets"].reshape((-1, 1))

    # use 15% of train data for testing
    xtr, xva, ytr, yva = train_test_split(xtr, ytr, test_size=0.15, random_state=50)

    if size >= 1.0 and size <=0:
        return xtr, ytr, xva, yva, xte, yte
    
    discard_size = 1.0 - size
    xtr, _, ytr, _ = train_test_split(xtr, ytr, test_size=discard_size, random_state=50)
    xva, _, yva, _ = train_test_split(xva, yva, test_size=discard_size, random_state=50)
    xte, _, yte, _ = train_test_split(xte, yte, test_size=discard_size, random_state=50)
    
    xtr, ytr = reshape_dataset(xtr, ytr.ravel())
    xva, yva = reshape_dataset(xva, yva.ravel())
    xte, yte = reshape_dataset(xte, yte.ravel())
    
#     print(xtr.shape)
#     print(ytr.shape)
    
#     print(xva.shape)
#     print(yva.shape)
    
#     print(xte.shape)
#     print(yte.shape)
    
    return xtr, ytr, xva, yva, xte, yte

def reshape_dataset(x, y):
    n_elem, n_feat = x.shape
    n_feat = int(n_feat**0.5)
    x = x.reshape((n_elem, n_feat, n_feat, 1))

    # make sure classes are between 0 and num_classes
    new_y = np.ones_like(y)*(-1)
    num_classes = np.unique(y)
    for idx, label in enumerate(num_classes):
        new_y[y == label] = idx
    
    y = to_categorical(new_y)
    
    return x, y
    
def load_clothes(size=1.0):
    return load_dataset("../data/clothes_train.npz", "../data/clothes_test.npz", size)
    
    
def load_faces(size=1.0):
    return load_dataset("../data/faces_train.npz", "../data/faces_test.npz", size)

# load_faces()
# load_clothes()

## Train

In [53]:
def save_plot_metrics(log_file_name, history):
    keys = history.history.keys()

    f, ax = plt.subplots(len(keys), 1, figsize=(5, 22))

    for idx, k in enumerate(keys):
        ax[idx].plot(history.history[k])
        ax[idx].set_title("model " + k)
        ax[idx].set_ylabel(k)
        ax[idx].set_xlabel('epoch')
    
    f.savefig("{:s}.png".format(log_file_name), dpi=90)

def save_log_metrics(log_file_name, hyper, history):
    header = ""

    for key in hyper:
        header = header + ", " + key + ": " + str(hyper[key])

    header = header[2:]

    with open(log_file_name + ".txt", "w+") as log_file:
        log_file.write(header+"\n")
        
        keys = history.history.keys()
        head = ""
        
        c = 0
        for k in keys:
            if c == 0:
                l = len(history.history[k]) # number of epochs
                h = np.zeros(l)
            head = head + k + ","
            h = np.vstack((h, history.history[k]))
            c = c + 1

        head = head[:-1]
        head = head + "\n"
        log_file.write(head)

        h = h[1:,:]
        h = h.T

        for row in h:
            new_line = ""
            for value in row:
                new_line = new_line + "{:.8f},".format(value)
            new_line = new_line[:-1]
            new_line = new_line + "\n"
            log_file.write(new_line)

    log_file.close()

def generator(X, y, batch_size):
    total_input = len(X)
    
    while True:
        features, targets = [], []
        i = 0
        while len(features) < batch_size:
            index = randrange(0, total_input)
            feats = X[index]
            labels = y[index]
           
            features.append(feats)
            targets.append(labels)
            
        yield (np.array(features), np.array(targets))

def getFeaturesTargets(X, y):
    feats = []
    targets = []

    for feat, label in zip(X, y):
        feats.append(feat)
        targets.append(label)

    return np.array(feats), np.array(targets)

# arg dataset is 0 for clothes, 1 for faces
def train_model(model, hyper_params, log_file_name, dataset=0, dataset_size=1.0):
    learning_rate = hyper_params["learning_rate"]
    training_size = hyper_params["training_size"]
    batch_size = hyper_params["batch_size"]
    num_epochs = hyper_params["num_epochs"]

    rng = np.random.RandomState(seed)

    #train_data, valid_data, test_data = load_data(rng, batch_size=hyper_params["batch_size"])
    if dataset == 0:
        xtr, ytr, xva, yva, xte, yte = load_clothes(dataset_size)
    else:
        xtr, ytr, xva, yva, xte, yte = load_faces(dataset_size)

    history = model.fit_generator(
        generator(xtr, ytr, batch_size),
        samples_per_epoch = training_size,
        validation_data = getFeaturesTargets(xva, yva),
        nb_epoch = num_epochs
        )

    eval_ = model.evaluate(xte, yte)
    for val, key in zip(eval_, model.metrics_names):
        hyper_params[key] = val

    save_log_metrics(log_file_name, hyper_params, history)
    save_plot_metrics(log_file_name, history)

In [55]:
hyper = OrderedDict()
hyper["learning_rate"] = 0.01
hyper["training_size"] = 2000
hyper["batch_size"] = 50
hyper["num_epochs"] = 10

model1 = get_model_1(hyper["learning_rate"])
model2 = get_model_2(hyper["learning_rate"])

/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, 5, kernel_initializer="random_uniform", bias_initializer="zeros", padding="valid")`
  del sys.path[0]
/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(5, 5, kernel_initializer="random_uniform", bias_initializer="zeros", padding="valid")`
/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, 5, kernel_initializer="random_uniform", bias_initializer="zeros", padding="valid")`


### Model1 with clothes

In [57]:
train_model(model1, hyper, "model1_clothes", dataset=0)

Epoch 1/10
  19/2000 [..............................] - ETA: 18s - loss: 0.1511 - acc: 0.9495

/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=(array([[[..., steps_per_epoch=2000, epochs=10)`


2000/2000 [==============================] - 19s - loss: 0.1712 - acc: 0.9440 - val_loss: 0.2643 - val_acc: 0.9296
Epoch 2/10
2000/2000 [==============================] - 18s - loss: 0.1721 - acc: 0.9440 - val_loss: 0.2209 - val_acc: 0.9307
Epoch 3/10
2000/2000 [==============================] - 19s - loss: 0.1666 - acc: 0.9459 - val_loss: 0.2003 - val_acc: 0.9418
Epoch 4/10
2000/2000 [==============================] - 17s - loss: 0.1721 - acc: 0.9441 - val_loss: 0.2077 - val_acc: 0.9342
Epoch 5/10
2000/2000 [==============================] - 17s - loss: 0.1695 - acc: 0.9446 - val_loss: 0.2163 - val_acc: 0.9376
Epoch 6/10
2000/2000 [==============================] - 17s - loss: 0.1710 - acc: 0.9441 - val_loss: 0.2159 - val_acc: 0.9387
Epoch 7/10
2000/2000 [==============================] - 18s - loss: 0.1612 - acc: 0.9476 - val_loss: 0.2441 - val_acc: 0.9189
Epoch 8/10
2000/2000 [==============================] - 22s - loss: 0.1741 - acc: 0.9432 - val_loss: 0.1979 - val_acc: 0.9382
Epo

### Model1 with faces

In [58]:
train_model(model1, hyper, "model1_faces", dataset=1)

Epoch 1/10
  19/2000 [..............................] - ETA: 19s - loss: 2.8665 - acc: 0.0821

/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=(array([[[..., steps_per_epoch=2000, epochs=10)`


2000/2000 [==============================] - 17s - loss: 1.4034 - acc: 0.3872 - val_loss: 1.3603 - val_acc: 0.3853
Epoch 2/10
2000/2000 [==============================] - 18s - loss: 1.3625 - acc: 0.3885 - val_loss: 1.3589 - val_acc: 0.3858
Epoch 3/10
2000/2000 [==============================] - 18s - loss: 1.3607 - acc: 0.3910 - val_loss: 1.3608 - val_acc: 0.3861
Epoch 4/10
2000/2000 [==============================] - 17s - loss: 1.3599 - acc: 0.3915 - val_loss: 1.3626 - val_acc: 0.3869
Epoch 5/10
2000/2000 [==============================] - 16s - loss: 1.3543 - acc: 0.3934 - val_loss: 1.3743 - val_acc: 0.3855
Epoch 6/10
2000/2000 [==============================] - 17s - loss: 1.3551 - acc: 0.3918 - val_loss: 1.3674 - val_acc: 0.3861
Epoch 7/10
2000/2000 [==============================] - 21s - loss: 1.3568 - acc: 0.3940 - val_loss: 1.3709 - val_acc: 0.3858
Epoch 8/10
2000/2000 [==============================] - 19s - loss: 1.3565 - acc: 0.3915 - val_loss: 1.3717 - val_acc: 0.3872
Epo

### Model2 with clothes

In [59]:
train_model(model2, hyper, "model2_clothes", dataset=0)

/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=(array([[[..., steps_per_epoch=2000, epochs=10)`


Epoch 1/10
2000/2000 [==============================] - 22s - loss: 0.1915 - acc: 0.9418 - val_loss: 0.1802 - val_acc: 0.9436
Epoch 2/10
2000/2000 [==============================] - 25s - loss: 0.1640 - acc: 0.9509 - val_loss: 0.1656 - val_acc: 0.9429
Epoch 3/10
2000/2000 [==============================] - 26s - loss: 0.1738 - acc: 0.9487 - val_loss: 0.1584 - val_acc: 0.9431
Epoch 4/10
2000/2000 [==============================] - 25s - loss: 0.1681 - acc: 0.9485 - val_loss: 0.2167 - val_acc: 0.9169
Epoch 5/10
2000/2000 [==============================] - 24s - loss: 0.1732 - acc: 0.9480 - val_loss: 0.2195 - val_acc: 0.9327
Epoch 6/10
2000/2000 [==============================] - 23s - loss: 0.1711 - acc: 0.9480 - val_loss: 0.1689 - val_acc: 0.9413
Epoch 7/10
2000/2000 [==============================] - 25s - loss: 0.1716 - acc: 0.9456 - val_loss: 0.2208 - val_acc: 0.9429
Epoch 8/10
2000/2000 [==============================] - 23s - loss: 0.1756 - acc: 0.9494 - val_loss: 0.1868 - val_acc:

### Model2 with faces

In [60]:
train_model(model2, hyper, "model2_faces", dataset=1)

Epoch 1/10
  16/2000 [..............................] - ETA: 23s - loss: 5.4483 - acc: 0.3100

/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: The semantics of the Keras 2 argument  `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Update your method calls accordingly.
/afs/inf.ed.ac.uk/user/s17/s1700260/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=(array([[[..., steps_per_epoch=2000, epochs=10)`


2000/2000 [==============================] - 23s - loss: 1.4226 - acc: 0.3778 - val_loss: 1.3649 - val_acc: 0.3889
Epoch 2/10
2000/2000 [==============================] - 20s - loss: 1.3630 - acc: 0.3880 - val_loss: 1.3749 - val_acc: 0.3889
Epoch 3/10
2000/2000 [==============================] - 19s - loss: 1.3652 - acc: 0.3884 - val_loss: 1.3712 - val_acc: 0.3906
Epoch 4/10
2000/2000 [==============================] - 21s - loss: 1.3614 - acc: 0.3916 - val_loss: 1.3758 - val_acc: 0.3881
Epoch 5/10
2000/2000 [==============================] - 24s - loss: 1.3629 - acc: 0.3912 - val_loss: 1.3717 - val_acc: 0.3892
Epoch 6/10
2000/2000 [==============================] - 23s - loss: 1.3573 - acc: 0.3926 - val_loss: 1.3648 - val_acc: 0.3886
Epoch 7/10
2000/2000 [==============================] - 23s - loss: 1.3582 - acc: 0.3916 - val_loss: 1.3841 - val_acc: 0.3883
Epoch 8/10
2000/2000 [==============================] - 21s - loss: 1.3570 - acc: 0.3951 - val_loss: 1.3759 - val_acc: 0.3872
Epo